In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "BiasedMNIST",
            'fairness_agg': 'mean',

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 5,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 128,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'lambda': 5,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.002
              }
    


#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_lambda={params['lambda']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=BiasedMNIST/seed=1_epoch=5_lr=0.001_alpha=0.002_lambda=5


In [2]:
"MNIST" in params['dataset']

True

In [3]:
from datasets import BiasedMNIST

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = min(params['per_task_examples'], 15000))

    n_feature = 3*28*28
    

noise = 0.0


[0 1 2 3 4 5 6 7 8 9]


In [4]:
from torchvision.transforms.functional import to_pil_image
from cl_gym.benchmarks.transforms import MNIST_MEAN, MNIST_STD
COLOR_MAP = {
    0: (1, 0, 0),
    1: (0, 1, 0),
    2: (1, 1, 0),
    3: (0, 0, 1),
    4: (1, 0.65, 0),
    5: (0.5, 0, 0.5),
    6: (0, 1, 1),
    7: (1, 0.75, 0.8),
    8: (0.8, 1, 0),
    9: (.588, .294, 0.)
}

r = (1 - 0.1913)
color_values = np.array(list(COLOR_MAP.values()))
m_rgb = color_values.mean(axis=0)
std_rgb = color_values.std(axis=0)

bmnist_mean = [r*m + MNIST_MEAN[0] for m in m_rgb]
bmnist_std = [(r*s**2+(1-r)*MNIST_STD[0]**2+r*(1-r)*(bmnist_mean[i] - m_rgb[i])**2)**0.5 for i, s in enumerate(std_rgb)]

unnormalize = torchvision.transforms.Normalize([-m/s for m, s in zip(bmnist_mean, bmnist_std)], [1/s for s in bmnist_std])
sample_idx = 95
to_pil_image(unnormalize(benchmark.trains[2].inputs[sample_idx]), mode="RGB")

In [5]:
target_label = 0
incremental_step = 1
# cat_img = torch.cat([img for img in benchmark.trains[incremental_step].inputs[benchmark.trains[incremental_step].targets == target_label][20:30]], dim=2)
cat_img = torch.cat([img for img in benchmark.tests[incremental_step].inputs[benchmark.tests[incremental_step].targets == target_label][20:30]], dim=2)
to_pil_image(unnormalize(cat_img), mode="RGB")

In [6]:
from trainers.fair_trainer import FairContinualTrainer
from metrics import FairMetricCollector
from algorithms.sensitive import Heuristic3

backbone = cl.backbones.MLP2Layers(input_dim=n_feature, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Heuristic3(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

# from trainers.FairContinualTrainer import FairContinualTrainer
# trainer = FairContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])

trainer = FairContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
if params['fairness_agg'] == "mean":
    agg = np.mean
elif params['fairness_agg'] == "max":
    agg = np.max
else:
    raise NotImplementedError

metric_manager_callback.meters['multiclass_eo'].agg = agg
metric_manager_callback.meters['multiclass_eo'].agg


<function mean at 0x7f43d76f72b0>

In [8]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 95.17935808684707, 'loss': 0.0008248412862737128, 'multiclass_eo': [0.056451612903225756, 0.13184931506849318], 'accuracy_s0': 100.0, 'accuracy_s1': 90.58495360141406, 'classwise_accuracy': {0: array([952, 980]), 1: array([1058, 1135])}}
[2] Eval metrics for task 1 >> {'accuracy': 97.99671851119302, 'loss': 0.00031524862627329273, 'multiclass_eo': [0.07191780821917804, 0.006048387096774244], 'accuracy_s0': 100.0, 'accuracy_s1': 96.10169023420238, 'classwise_accuracy': {1: array([1093, 1135]), 0: array([977, 980])}}
[3] Eval metrics for task 1 >> {'accuracy': 99.75186550391082, 'loss': 0.00018061498096366864, 'multiclass_eo': [0.008064516129032251, 0.001712328767123239], 'accuracy_s0': 100.0, 'accuracy_s1': 99.51115775519223, 'classwise_accuracy': {0: array([976, 980]), 1: array([1134, 1135])}}
[4] Eval metrics for task 1 >> {'accuracy': 99.75186550391082, 'loss': 0.000133821875491040

In [9]:
metric_manager_callback.meters['accuracy'].get_data()

array([[99.803,  0.   ,  0.   ,  0.   ,  0.   ],
       [93.605, 71.675,  0.   ,  0.   ,  0.   ],
       [86.908, 81.792, 85.599,  0.   ,  0.   ],
       [85.296, 73.513, 84.153, 68.08 ,  0.   ],
       [79.566, 70.281, 72.788, 80.242, 70.494]])

In [10]:
[np.round(x, 3) for x in metric_manager_callback.meters['accuracy'].compute_overall()]

[99.803, 82.64, 84.766, 77.761, 74.674]

In [11]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

83.92881851324503

In [12]:
[np.round(x, 3) for x in metric_manager_callback.meters['multiclass_eo'].compute_overall()]

[0.004, 0.2, 0.251, 0.35, 0.423]

In [13]:
np.mean(metric_manager_callback.meters['multiclass_eo'].compute_overall())

0.24584230865397574

In [14]:
incremental_step = 2

print(f"{incremental_step=}")
print(f"sensitive samples / all samples = {(benchmark.trains[incremental_step].sensitive != benchmark.trains[incremental_step].targets).sum().item()} / {benchmark.trains[incremental_step].sensitive.shape[0]}")
one_idx = benchmark.trains[incremental_step].sample_weight > 0.9
print(f"sensitive samples / selected samples = {(benchmark.trains[incremental_step].sensitive[one_idx] != benchmark.trains[incremental_step].targets[one_idx]).sum().item()} / {one_idx.sum().item()}")


incremental_step=2
sensitive samples / all samples = 2194 / 12089
sensitive samples / selected samples = 1265 / 4017


In [15]:
step_class = 2
for i in range(2, 6):
    incremental_step = i
    print(f"{incremental_step=}")
    one_idx = benchmark.trains[incremental_step].sample_weight > 0.9

    print(f"{2*i-2} : {(benchmark.trains[incremental_step].targets == (2*i-2)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-2)).sum().item()}")
    print(f"{2*i-1} : {(benchmark.trains[incremental_step].targets == (2*i-1)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-1)).sum().item()}")


incremental_step=2
2 : 5958 --> 2118
3 : 6131 --> 1899
incremental_step=3
4 : 5842 --> 2403
5 : 5421 --> 1772
incremental_step=4
6 : 5918 --> 1908
7 : 6265 --> 1854
incremental_step=5
8 : 5851 --> 2019
9 : 5949 --> 1456


In [16]:
for eos in metric_manager_callback.meters['multiclass_eo'].get_data():
    for eo in eos:
        print(round(eo, 3),end=" ")
    print()

0.006 0.002 
0.123 0.109 0.244 0.325 
0.26 0.22 0.265 0.433 0.172 0.159 
0.254 0.254 0.475 0.36 0.285 0.271 0.901 0.005 
0.342 0.415 0.56 0.395 0.496 0.486 0.388 0.354 0.747 0.049 


In [17]:
metric_manager_callback.meters['classwise_accuracy'].get_data()

[{0: array([977, 980]), 1: array([1134, 1135])},
 {1: array([1057, 1135]),
  0: array([922, 980]),
  3: array([ 752, 1010]),
  2: array([ 711, 1032])},
 {1: array([ 971, 1135]),
  0: array([865, 980]),
  2: array([ 882, 1032]),
  3: array([ 789, 1010]),
  4: array([861, 982]),
  5: array([745, 892])},
 {1: array([ 968, 1135]),
  0: array([836, 980]),
  3: array([ 749, 1010]),
  2: array([ 752, 1032]),
  4: array([837, 982]),
  5: array([741, 892]),
  7: array([ 464, 1028]),
  6: array([872, 958])},
 {1: array([ 919, 1135]),
  0: array([766, 980]),
  3: array([ 670, 1010]),
  2: array([ 766, 1032]),
  4: array([736, 982]),
  5: array([630, 892]),
  7: array([ 816, 1028]),
  6: array([777, 958]),
  9: array([ 599, 1009]),
  8: array([795, 974])}]